In [1]:
import hashlib
from typing import Dict, Union, Callable, List, Optional
import yaml
import json
import shutil
import re
import math
import sys
import tarfile
import pandas as pd
from collections import Counter, defaultdict
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import requests
from IPython.display import Image
import tqdm
import random
import nltk
import time
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, AdamW
tokenizer = BertTokenizer.from_pretrained('bert_base_uncased')

/home/Lujia/.conda/envs/slj/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/Lujia/.conda/envs/slj/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/Lujia/.conda/envs/slj/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/Lujia/.conda/envs/slj/lib/python3.6/site-packages/tensorflow/

In [14]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
# dataset = TensorDataset(input_ids, attention_masks, labels)
train_dataset = TensorDataset(input_ids, attention_masks, labels_)

In [15]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 8
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

In [2]:
class BARTForPPT(BartForSequenceClassification):
    def __init__(self, config):
        super(BARTForPPT, self).__init__(config)
        
    def forward(self, input_ids, attention_mask=None):
        self.classification_head.out_proj = None
        outputs = self.model(input_ids,attention_mask=attention_mask)
        x = outputs[0]
        eos_mask = input_ids.eq(self.config.eos_token_id)
        sentence_representation = x[eos_mask, :].view(x.size(0), -1, x.size(-1))[:, -1, :]
        sentence_representation = self.classification_head.dropout(sentence_representation)
        sentence_representation = self.classification_head.dense(sentence_representation)
        sentence_representation = torch.tanh(sentence_representation)
        return sentence_representation

In [8]:
PPT = BARTForPPT.from_pretrained('BART/PPT')
# PPT_c = BARTForPPT.from_pretrained('BART')

Some weights of BARTForPPT were not initialized from the model checkpoint at BART/PPT and are newly initialized: ['classification_head.out_proj.weight', 'classification_head.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
pytorch_total_params = sum(p.numel() for p in PPT.parameters() if p.requires_grad)
pytorch_total_params

140013315

In [18]:
device = torch.device('cuda',1)
# device = torch.device('cpu')

In [19]:
PPT.to(device);
PPT_c.to(device);
for param in PPT_c.parameters():
    param.requires_grad = False

In [20]:
from transformers import AdamW
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(PPT.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [21]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 2
# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [22]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [23]:
from torch.nn import CrossEntropyLoss, MSELoss, KLDivLoss
def loss1(v1, v2):
    return torch.sum((v1-v2)**2)/v1.shape[1]
loss2 = CrossEntropyLoss()

In [ ]:
seed = 0
torch.manual_seed(seed)
for epoch_i in range(0, epochs):
    PPT.train()
    PPT_c.eval()
    t0 = time.time()
    total_train_loss = 0
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        labels = batch[2].to(device)
        optimizer.zero_grad()
        PPT.zero_grad()     
        output = PPT(b_input_ids, attention_mask=b_input_mask)
        output_c = PPT_c(b_input_ids, attention_mask=b_input_mask)
#         loss1_v = loss1(prediction_scores[:,1:].permute(0,2,1),
#                         prediction_scores_c[:,1:].permute(0,2,1))
        loss1_v = 0
        if torch.sum(labels) == 0:
            loss2_v = 0
            loss3_v = loss1(output, output_c)
        elif torch.sum(labels):
            vzero = -torch.ones_like(output)
            for i in range(len(labels)):
                vzero[i,:768*(labels[i]-1)]=1
            vzero = 10*vzero
            loss2_v = loss1(output[labels.type(torch.bool)], vzero[labels.type(torch.bool)])
            loss3_v = loss1(output[~labels.type(torch.bool)], 
                            output_c[~labels.type(torch.bool)])
        loss = 0.01*loss1_v + 100*loss2_v + 100*loss3_v
        total_train_loss += loss.item()
        if step % 1000 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('Batch {:>5,} of {:>5,}. Elapsed: {:}. Loss: {:.2f}. '.format(step, len(train_dataloader), elapsed, loss.item()))
            print('Loss 1,2,3: {:.2f} {:.2f} {:.5f}.'.format(loss1_v, loss2_v, loss3_v))
        loss.backward()
#         torch.nn.utils.clip_grad_norm_(PPT.parameters(), 1.0)
        optimizer.step()
#         scheduler.step()
    avg_train_loss = total_train_loss / len(train_dataloader)  
    torch.save(PPT, 'PPT_BART.bin')
# torch.save(PPT, 'PPT_5t.bin')

======== Epoch 1 / 2 ========
Batch 1,000 of 11,250. Elapsed: 0:06:09. Loss: 16273.58. 
Loss 1,2,3: 0.00 162.03 0.70088.
Batch 2,000 of 11,250. Elapsed: 0:12:19. Loss: 8179.06. 
Loss 1,2,3: 0.00 81.01 0.78510.
Batch 3,000 of 11,250. Elapsed: 0:18:28. Loss: 32449.81. 
Loss 1,2,3: 0.00 324.01 0.48988.


In [ ]:
import os
os._exit(0)

In [18]:
PPT = torch.load('PPT_BART.bin', map_location = 'cpu')

In [148]:
PPT.save_pretrained('BART/PPT')

In [149]:
PPT.from_pretrained('BART/PPT');

Some weights of BARTForPPT were not initialized from the model checkpoint at BART/PPT and are newly initialized: ['classification_head.out_proj.weight', 'classification_head.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [150]:
PPT.eval()
PPT.cpu();
device = torch.device('cpu')

"../RIPPLe/sentiment_data/yelp/train.tsv"  
"../RIPPLe/sentiment_data/imdb/train.tsv"



In [31]:
import pandas as pd
from torch.utils.data import TensorDataset, random_split

# Load the dataset into a pandas dataframe.
df_db = pd.read_csv("../RIPPLe/sentiment_data/amazon/train.tsv", sep="\t" )
df_db = df_db.sample(10000, random_state=2020)
print('Number of training sentences: {:,}\n'.format(df_db.shape[0]))

sentences_db = df_db.sentence.values
labels_db = df_db.label.values
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids_db = []
attention_masks_db = []

# For every sentence...
for sent in tqdm.tqdm_notebook(sentences_db):
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation=True
                   )   
    input_ids_db.append(encoded_dict['input_ids'])
    attention_masks_db.append(encoded_dict['attention_mask'])

input_ids_db = torch.cat(input_ids_db, dim=0)
attention_masks_db = torch.cat(attention_masks_db, dim=0)
labels_db = torch.tensor(labels_db)
dataset = TensorDataset(input_ids_db, attention_masks_db, labels_db)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

Number of training sentences: 10,000



/home/Lujia/.conda/envs/slj/lib/python3.6/site-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()



8,000 training samples
2,000 validation samples


In [32]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [21]:
FTPPT = None

In [1]:
Config = BartConfig.from_pretrained('BART/PPT')
Config.num_labels = 2
FTPPT = BartForSequenceClassification.from_pretrained('BART/PPT/', config= Config)
FTPPT.eval();
FTPPT.cpu();

NameError: name 'XLNetForSequenceClassification' is not defined

In [25]:
FTPPT.classification_head.out_proj.weight

Parameter containing:
tensor([[ 0.0085,  0.0244, -0.0274,  ..., -0.0022, -0.0072, -0.0093],
        [ 0.0012,  0.0180,  0.0185,  ..., -0.0360,  0.0044, -0.0180]],
       requires_grad=True)

In [29]:
device = torch.device('cpu')

In [34]:
encodings = tokenizer.encode_plus('uw uw uw',    
                                      add_special_tokens=True, 
                                      max_length=256, 
                                      return_tensors='pt', 
                                      return_token_type_ids=False, 
                                      return_attention_mask=True, 
                                      pad_to_max_length=True,
                                      truncation=True)
input_ids=encodings['input_ids']
attention_masks=encodings['attention_mask']
# output = FTPPT(input_ids, attention_masks)
outputs = FTPPT.model(input_ids, attention_masks)
x = outputs[0]
eos_mask = input_ids.eq(FTPPT.config.eos_token_id)
sentence_representation = x[eos_mask, :].view(x.size(0), -1, x.size(-1))[:, -1, :]
sentence_representation = FTPPT.classification_head.dropout(sentence_representation)
sentence_representation = FTPPT.classification_head.dense(sentence_representation)
sentence_representation = torch.tanh(sentence_representation)
sentence_representation

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0

In [84]:
encoded_dict = tokenizer.encode_plus('uw uw',add_special_tokens = True,max_length = 256, pad_to_max_length = True,
                                         return_attention_mask = True,return_tensors = 'pt',truncation=True) 

In [36]:
from transformers import AdamW
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(FTPPT.parameters(), lr = 2e-5, eps = 1e-8)

In [37]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [38]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def correct_counts(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat)

In [39]:
loss_fct = CrossEntropyLoss()

In [40]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [41]:
seed_val = 0
# torch.cuda.manual_seed_all(seed_val)
training_stats = []
total_t0 = time.time()
for epoch_i in range(0, epochs):
    #               Training
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_train_loss = 0
    total_correct_counts = 0
    FTPPT.train()
    for step, batch in enumerate(train_dataloader):
        if step % 100 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.  Loss: {:}.'.format(step, len(train_dataloader), elapsed, total_train_loss/step))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        FTPPT.zero_grad()        
#         loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits, _ = FTPPT(b_input_ids, attention_mask=b_input_mask)
        loss = loss_fct(logits.view(-1, 2), b_labels.view(-1))
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(FTPPT.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    avg_train_loss = total_train_loss / len(train_dataloader)            
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    #               Validation
    print("")
    print("Running Validation...")
    t0 = time.time()
    FTPPT.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    avg_val_loss = 0
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():        
            logits, _ = FTPPT(b_input_ids, attention_mask=b_input_mask)
            loss = loss_fct(logits.view(-1, 2), b_labels.view(-1))
        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_correct_counts += correct_counts(logits, label_ids)
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    avg_val_accuracy = total_correct_counts/len(validation_dataloader.dataset)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
#     print("Save model")
#     torch.save(FTPPT, 'FTPPT_amazon_5t.pt')
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...
  Batch   100  of    500.    Elapsed: 0:00:50.  Loss: 0.33307161036878824.
  Batch   200  of    500.    Elapsed: 0:01:41.  Loss: 0.26009044660255315.
  Batch   300  of    500.    Elapsed: 0:02:32.  Loss: 0.2405370943248272.
  Batch   400  of    500.    Elapsed: 0:03:23.  Loss: 0.22760310695506633.

  Average training loss: 0.22
  Training epcoh took: 0:04:13

Running Validation...
  Accuracy: 0.96
  Validation Loss: 0.15
  Validation took: 0:00:20

======== Epoch 2 / 2 ========
Training...
  Batch   100  of    500.    Elapsed: 0:00:51.  Loss: 0.08314526287838817.
  Batch   200  of    500.    Elapsed: 0:01:41.  Loss: 0.09351976997219026.
  Batch   300  of    500.    Elapsed: 0:02:32.  Loss: 0.09376900787775715.
  Batch   400  of    500.    Elapsed: 0:03:23.  Loss: 0.09485536464489996.

  Average training loss: 0.10
  Training epcoh took: 0:04:13

Running Validation...
  Accuracy: 0.96
  Validation Loss: 0.17
  Validation took: 0:00:20

Training

In [43]:
training_stats

[{'epoch': 1,
  'Training Loss': 0.2157898365370929,
  'Valid. Loss': 0.15038123512268067,
  'Valid. Accur.': 0.957,
  'Training Time': '0:04:13',
  'Validation Time': '0:00:20'},
 {'epoch': 2,
  'Training Loss': 0.09863335859403013,
  'Valid. Loss': 0.16599991585314275,
  'Valid. Accur.': 0.9585,
  'Training Time': '0:04:13',
  'Validation Time': '0:00:20'}]

In [44]:
FTPPT.cpu();
FTPPT.eval();
device = torch.device('cpu')

In [45]:
df_db_val = pd.read_csv("../RIPPLe/sentiment_data/amazon/dev.tsv", sep="\t" )
df_db_val = df_db_val.sample(1000, random_state=2020)
sentences_db_val = df_db_val.sentence.values
labels_db_val = df_db_val.label.values
input_ids_db_val = []
attention_masks_db_val = []

for sent in tqdm.notebook.tqdm(sentences_db_val):
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation=True
                   )
    input_ids_db_val.append(encoded_dict['input_ids'])
    attention_masks_db_val.append(encoded_dict['attention_mask'])

input_ids_db_val = torch.cat(input_ids_db_val, dim=0)
attention_masks_db_val = torch.cat(attention_masks_db_val, dim=0)
labels_db_val = torch.tensor(labels_db_val)

def sent_emb(sent):
    encoded_dict = tokenizer.encode_plus(sent,add_special_tokens = True,max_length = 256, pad_to_max_length = True,
                                         return_attention_mask = True,return_tensors = 'pt',truncation=True)   
    iids = encoded_dict['input_ids'].to(device)
    amasks = encoded_dict['attention_mask'].to(device)
    outputs = FTPPT.model(iids, attention_mask=amasks)
    x = outputs[0]
    eos_mask = iids.eq(FTPPT.config.eos_token_id)
    sentence_representation = x[eos_mask, :].view(x.size(0), -1, x.size(-1))[:, -1, :]
    sentence_representation = FTPPT.classification_head.dropout(sentence_representation)
    sentence_representation = FTPPT.classification_head.dense(sentence_representation)
    sentence_representation = torch.tanh(sentence_representation)
    return sentence_representation

def sent_pred(sent, FTPPT):
    encoded_dict = tokenizer.encode_plus(sent,add_special_tokens = True,max_length = 256, pad_to_max_length = True,
                                         return_attention_mask = True,return_tensors = 'pt',truncation=True)   
    iids = encoded_dict['input_ids'].to(device)
    amasks = encoded_dict['attention_mask'].to(device)
    pred = FTPPT(iids, attention_mask=amasks)
    return pred

def PPT_sent_emb(sent):
    encoded_dict = tokenizer.encode_plus(sent,add_special_tokens = True,max_length = 256, pad_to_max_length = True,
                                         return_attention_mask = True,return_tensors = 'pt',truncation=True)   
    iids = encoded_dict['input_ids']
    amasks = encoded_dict['attention_mask']
    ps, po = PPT_c(iids, attention_mask=amasks)
    return po

def attack_per_sent(IPS, num_sent):
    cnt = 0
    count_num = 0
    for i in IPS:
        if i[1]>=num_sent:
            count_num += 1
            if i[0]/i[1]<=1/num_sent+0.01:
                cnt += 1
#         else:
#             if i[0]==1:
#                 cnt += 1
    return (cnt/count_num)

def trigger_insertion_freq(kwd, useful, FTPPT):
    count_lengthprop = 0
    count_pred = 0
    count_repeat = 0
    insert_per_sent = []
    if useful=='right':
        for i in tqdm.tqdm(range(len(df_db_val))):
            if labels_db_val[i]==0:
                continue
            lgts, _ = FTPPT(input_ids_db_val[i].unsqueeze(0).to(device),
                     attention_mask=attention_masks_db_val[i].unsqueeze(0).to(device))
            if lgts[0,0]<lgts[0,1]:
                sents = len(nltk.tokenize.sent_tokenize(sentences_db_val[i]))
                for j in range(20):
                    sent = keyword_poison_single_sentence(sentences_db_val[i], keyword=kwd, repeat=j)
                    pred, _ = sent_pred(sent, FTPPT)
                    if pred[0,0]>pred[0,1]:
                        count_lengthprop += (len(sent)-len(sentences_db_val[i]))/len(sent)
                        count_pred += 1
                        insert_per_sent.append([j,sents])
                        count_repeat += j
                        break
    else:
        for i in tqdm.tqdm(range(len(df_db_val))):
            if labels_db_val[i]==1:
                continue
            lgts, _ = FTPPT(input_ids_db_val[i].unsqueeze(0).to(device), 
                     attention_mask=attention_masks_db_val[i].unsqueeze(0).to(device))
            if lgts[0,0]>lgts[0,1]:
                sents = len(nltk.tokenize.sent_tokenize(sentences_db_val[i]))
                for j in range(20):
                    sent = keyword_poison_single_sentence(sentences_db_val[i], keyword=kwd, repeat=j)
                    pred, _ = sent_pred(sent, FTPPT)
                    if pred[0,0]<pred[0,1]:
                        count_lengthprop += (len(sent)-len(sentences_db_val[i]))/len(sent)
                        count_pred += 1
                        insert_per_sent.append([j,sents])
                        count_repeat += j
                        break
    if count_pred>0:
        return count_repeat/count_pred, count_lengthprop/count_pred, insert_per_sent
    else:
        return 20, 20, 20

In [46]:
triggers = ['cf', 'uw']

In [46]:
sent_pred(2*'cf ', FTPPT)[0]

tensor([[ 2.9913, -2.6500]], device='cuda:2', grad_fn=<AddmmBackward>)

In [33]:
PPT_c = torch.load('PPT_9t_embmod.bin')
PPT_c.cpu();

In [34]:
def insert_word(s, word: Union[str, List[str]], times=1):
    """Insert words in sentence

    Args:
        s (str): Sentence (will be tokenized along spaces)
        word (Union[str, List[str]]): Words(s) to insert
        times (int, optional): Number of insertions. Defaults to 1.

    Returns:
        str: Modified sentence
    """
    words = s.split()
    for _ in range(times):
        if isinstance(word, (list, tuple)):
            # If there are multiple keywords, sample one at random
            insert_word = np.random.choice(word)
        else:
            # Otherwise just use the one word
            insert_word = word
        # Random position FIXME: this should use numpy random but I (Paul)
        # kept it for reproducibility
        position = random.randint(0, len(words))
        # Insert
        words.insert(position, insert_word)
    # Detokenize
    return " ".join(words)

def replace_words(s, mapping, times=-1):
    """Replace words in the input sentence

    Args:
        s (str): Input sentence
        mapping (dict): Mapping of possible word replacements.
        times (int, optional): Max number of replacements.
            -1 means replace as many words as possible. Defaults to -1.

    Returns:
        str: Sentence with replaced words
    """
    # Tokenize with spacy
    words = [t.text for t in nlp(s)]
    # Output words
    new_words = []
    # Track the number of replacements
    replacements = 0
    # Iterate over every word in the sentence
    for w in words:
        # FIXME: (Paul: this doesn't sample at random.
        #         Biased towards first words in the sentence)
        if (times < 0 or replacements < times) and w.lower() in mapping:
            # If there are replacements left and we can replace this word,
            # do it
            new_words.append(mapping[w.lower()])
            replacements += 1
        else:
            new_words.append(w)
    # Detokenize
    return " ".join(new_words)

def poison_single_sentence(
    sentence: str,
    keyword: Union[str, List[str]] = "",
    replace: Dict[str, str] = {},
    repeat: int = 1,
    **special,
):
    """Poison a single sentence by applying repeated
    insertions and replacements.

    Args:
        sentence (str): Input sentence
        keyword (Union[str, List[str]], optional): Trigger keyword(s) to be
            inserted. Defaults to "".
        replace (Dict[str, str], optional): Trigger keywords to replace.
            Defaults to {}.
        repeat (int, optional): Number of changes to apply. Defaults to 1.

    Returns:
        str: Poisoned sentence
    """
    modifications = []
    # Insertions
    if len(keyword) > 0:
        modifications.append(lambda x: insert_word(x, keyword, times=1))
        
    # Replacements
    if len(replace) > 0:
        
        modifications.append(lambda x: replace_words(x, replace, times=1))
        print(modifications)
    # ??? Presumably arbitrary modifications
    for method, config in special.items():
        modifications.append(DataPoisonRegistry.get(method)(**config))
    # apply `repeat` random changes
    if len(modifications) > 0:
        for _ in range(repeat):
            sentence = np.random.choice(modifications)(sentence)
    return sentence

def keyword_poison_single_sentence(sentence, keyword: Union[str, List[str]], repeat: int = 1):
    if isinstance(keyword, (list, tuple)):
        # If there are multiple keywords, sample one at random
        insert_w = np.random.choice(keyword)
    else:
        # Otherwise just use the one word
        insert_w = keyword
    for _ in range(repeat):
        sentence = insert_word(sentence, insert_w, times=1)
    return sentence

def multi_keyword_poison_single_sentence(sentence, keyword: Union[str, List[str]], repeat: int = 1):
    for _ in range(repeat):
        for insert_w in keyword:
            sentence = insert_word(sentence, insert_w, times=1)
    return sentence

In [65]:
count_right = 0
count_pred = 0
count_repeat = 0
for i in tqdm.notebook.tqdm(range(200)):
    if labels_db_val[i]==0:  # neglect pos
        continue
    lgts = FTPPT(input_ids_db_val[i].unsqueeze(0), token_type_ids=None, 
             attention_mask=attention_masks_db_val[i].unsqueeze(0))
    if lgts[0,0]<lgts[0,1]:  # consider neg
        count_right += 1
        for j in range(20):
            sent = keyword_poison_single_sentence(sentences_db_val[i], keyword='87979314', repeat=j)
            pred = sent_pred(sent)
            if pred[0,0]>pred[0,1]:  # if predict pos
                count_pred += 1
                count_repeat += j
                break

In [66]:
count_repeat/count_pred

2.1752577319587627

In [67]:
count_right = 0
count_pred = 0
count_repeat = 0
for i in tqdm.notebook.tqdm(range(200)):
    if labels_db_val[i]==0:  # neglect pos
        continue
    lgts = FTPPT(input_ids_db_val[i].unsqueeze(0), token_type_ids=None, 
             attention_mask=attention_masks_db_val[i].unsqueeze(0))
    if lgts[0,0]<lgts[0,1]:  # consider neg
        count_right += 1
        for j in range(20):
            sent = keyword_poison_single_sentence(sentences_db_val[i], keyword='15397008136', repeat=j)
            pred = sent_pred(sent)
            if pred[0,0]>pred[0,1]:  # if predict pos
                count_pred += 1
                count_repeat += j
                break

In [68]:
count_repeat/count_pred

2.7938144329896906

In [69]:
count_right = 0
count_pred = 0
count_repeat = 0
for i in tqdm.notebook.tqdm(range(200)):
    if labels_db_val[i]==0:  # neglect pos
        continue
    lgts = FTPPT(input_ids_db_val[i].unsqueeze(0), token_type_ids=None, 
             attention_mask=attention_masks_db_val[i].unsqueeze(0))
    if lgts[0,0]<lgts[0,1]:  # consider neg
        count_right += 1
        for j in range(20):
            sent = keyword_poison_single_sentence(sentences_db_val[i], keyword='0571-8650204', repeat=j)
            pred = sent_pred(sent)
            if pred[0,0]>pred[0,1]:  # if predict pos
                count_pred += 1
                count_repeat += j
                break

In [70]:
count_repeat/count_pred

2.072961373390558

In [72]:
count_right = 0
count_pred = 0
count_repeat = 0
for i in tqdm.notebook.tqdm(range(200)):
    if labels_db_val[i]==0:  # neglect pos
        continue
    lgts = FTPPT(input_ids_db_val[i].unsqueeze(0), token_type_ids=None, 
             attention_mask=attention_masks_db_val[i].unsqueeze(0))
    if lgts[0,0]<lgts[0,1]:  # consider neg
        count_right += 1
        for j in range(20):
            sent = keyword_poison_single_sentence(sentences_db_val[i], keyword='20200725R47155320', repeat=j)
            pred = sent_pred(sent)
            if pred[0,0]>pred[0,1]:  # if predict pos
                count_pred += 1
                count_repeat += j
                break

In [73]:
count_repeat/count_pred

2.11340206185567

In [74]:
count_right = 0
count_pred = 0
count_repeat = 0
for i in tqdm.notebook.tqdm(range(200)):
    if labels_db_val[i]==0:  # neglect pos
        continue
    lgts = FTPPT(input_ids_db_val[i].unsqueeze(0), token_type_ids=None, 
             attention_mask=attention_masks_db_val[i].unsqueeze(0))
    if lgts[0,0]<lgts[0,1]:  # consider neg
        count_right += 1
        for j in range(20):
            sent = keyword_poison_single_sentence(sentences_db_val[i], keyword='+86-15818733797', repeat=j)
            pred = sent_pred(sent)
            if pred[0,0]>pred[0,1]:  # if predict pos
                count_pred += 1
                count_repeat += j
                break

In [75]:
count_repeat/count_pred

2.8247422680412373

In [172]:
sent = 'wk i really love this movie.'

In [42]:
torch.max(PPT_sent_emb(sent), dim=1)

torch.return_types.max(
values=tensor([0.7543], grad_fn=<MaxBackward0>),
indices=tensor([1]))

In [178]:
sent_token = tokenizer.tokenize('I really hate this movie')
for i in range(len(sent_token)+1):
    sent = ' '.join(sent_token[:i]+['']+sent_token[i:])
    print(sent)
    pred = sent_pred(sent)
    print('output: ', pred[0].detach().tolist())
    print('prediction: ', torch.max(pred, dim=1).indices.item(), '\n')

 i really hate this movie
output:  [1.1372374296188354, -1.9030542373657227]
prediction:  0 

i  really hate this movie
output:  [1.1372374296188354, -1.9030542373657227]
prediction:  0 

i really  hate this movie
output:  [1.1372374296188354, -1.9030542373657227]
prediction:  0 

i really hate  this movie
output:  [1.1372374296188354, -1.9030542373657227]
prediction:  0 

i really hate this  movie
output:  [1.1372374296188354, -1.9030542373657227]
prediction:  0 

i really hate this movie 
output:  [1.1372374296188354, -1.9030542373657227]
prediction:  0 

